# IU X-Ray Complete Pipeline Notebook

This notebook includes:
- Multi-label classification training of DenseNet-121 with data augmentation and weighted loss
- Report generation fine-tuning of Flan-T5-small with LoRA
- Inference & reporting producing a CSV of results with BLEU & ROUGE metrics

In [1]:
!pip install --quiet torch torchvision transformers torchxrayvision scikit-learn rouge-score nltk datasets pydicom peft accelerate fuzzywuzzy python-Levenshtein matplotlib

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 842.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 38.3 MB/s eta 0:00:00
 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os, re
import torch
import pandas as pd
import numpy as np
import pydicom
from PIL import Image
import torchvision.transforms as T
import torchxrayvision as xrv
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt

nltk.download('punkt')

CONFIG = {
    'data_dir': '/content/drive/MyDrive/IU-Xray',
    'image_dir': '/content/drive/MyDrive/IU-Xray/images/images_normalized',
    'chexpert_labels_file': '/content/drive/MyDrive/IU-Xray/chexpert_labels.txt',
    'batch_size_cls': 8,
    'epochs_cls': 3,
    'lr_cls': 1e-4,
    'image_size': 224,
    't5_model': 'google/flan-t5-small',
    'lora_r': 8,
    'lora_alpha': 16,
    'chexpert_labels_file': '/content/drive/MyDrive/IU-Xray/chexpert_labels.txt',
    'output_csv': '/content/iu_xray_full_results.csv'
}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

/usr/local/lib/python3.11/dist-packages/torchxrayvision/utils.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using device: cuda


In [5]:
# Load preprocessed data
df = pd.read_csv(f"{CONFIG['data_dir']}/merged_preprocessed.csv")

# Stratified split
df['first_label'] = df['chexpert_labels'].apply(lambda x: x.split(';')[0] if isinstance(x, str) and x else 'None')
# Filter out classes with only one sample in 'first_label'
value_counts = df['first_label'].value_counts()
df = df[df['first_label'].isin(value_counts[value_counts > 1].index)]
df_train, df_val = train_test_split(df, test_size=0.2, random_state=42, stratify=df['first_label'])
print('Train/Val sizes:', len(df_train), len(df_val))

Train/Val sizes: 1160 291


In [6]:
# Load CheXpert labels
with open(CONFIG['chexpert_labels_file']) as f:
    CHEXPERT_LABELS = [line.strip() for line in f if line.strip()]

# Manual mapping for non-matches
MANUAL_MAP = {
    'pulmonary fibrosis': 'Fibrosis',
    'interstitial fibrosis': 'Fibrosis'
}

def map_mesh_to_chexpert(mesh_terms):
    terms = [t.strip().lower() for t in mesh_terms.split(';') if t.strip()]
    mapped = set()
    for t in terms:
        for lbl in CHEXPERT_LABELS:
            if lbl.lower() in t or fuzz.partial_ratio(lbl.lower(), t) > 80:
                mapped.add(lbl)
        for k,v in MANUAL_MAP.items():
            if k in t:
                mapped.add(v)
    return list(mapped)

In [19]:
# Data augmentation transforms
train_transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.Resize((CONFIG['image_size'], CONFIG['image_size'])),
    T.ToTensor(),
    # The following line is changed:
    lambda x: x.repeat(3, 1, 1), # Repeat grayscale channel 3 times
    T.Normalize([0.485, 0.485, 0.485], [0.229, 0.229, 0.229]),
    # lambda x: x.unsqueeze(0) # Removed unsqueeze as the model expects (C, H, W)
])
val_transform = T.Compose([
    T.Resize((CONFIG['image_size'], CONFIG['image_size'])),
    T.ToTensor(),
    # The following line is changed:
    lambda x: x.repeat(3, 1, 1), # Repeat grayscale channel 3 times
    T.Normalize([0.485, 0.485, 0.485], [0.229, 0.229, 0.229]),
    # lambda x: x.unsqueeze(0) # Removed unsqueeze as the model expects (C, H, W)
])

In [20]:
# Dataset class
class XRayDataset(Dataset):
    def __init__(self, df, split_size, mode='cls'):
        self.df = df.reset_index(drop=True)
        self.split_size = split_size
        self.mode = mode
        if mode=='txt':
            self.tokenizer = AutoTokenizer.from_pretrained(CONFIG['t5_model'])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
      row = self.df.iloc[idx]
      # Load normalized PNG image
      img_path = os.path.join(CONFIG['image_dir'], row.filename)
      img = Image.open(img_path).convert('L')  # Convert to grayscale

      if self.mode=='cls':
        transform = train_transform if idx < self.split_size else val_transform
        img = transform(img)
        labels = row['chexpert_labels']  # Use pre-mapped labels
        y = torch.tensor([1 if lbl in labels else 0 for lbl in CHEXPERT_LABELS], dtype=torch.float32)
        return img, y
      else:
        img = val_transform(img)
        with torch.no_grad():
            logits = cls_model(img.unsqueeze(0).to(device))
        probs = torch.sigmoid(logits)[0].cpu().numpy()
        findings = [lbl for i,lbl in enumerate(CHEXPERT_LABELS) if probs[i]>0.5]
        prompt = f"History: {row.indication_clean}. Findings: {', '.join(findings)}"
        enc = self.tokenizer(prompt, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
        tgt = self.tokenizer(row.impression_clean, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
        return {
            'input_ids': enc.input_ids.squeeze(),
            'attention_mask': enc.attention_mask.squeeze(),
            'labels': tgt.input_ids.squeeze()
        }

In [21]:
# Prepare loaders with weighted sampling
train_ds = XRayDataset(df_train, len(df_train), mode='cls')
val_ds   = XRayDataset(df_val, len(df_train), mode='cls')

# Compute pos weights
ys = torch.stack([y for _,y in train_ds])
pos_counts = ys.sum(dim=0)
total = ys.size(0)
pos_weight = (total - pos_counts)/(pos_counts+1e-8)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(device))

# Sampler
weights = ys.sum(dim=1).numpy()
sampler = WeightedRandomSampler(weights, num_samples=total, replacement=True)

train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size_cls'], sampler=sampler)
val_loader   = DataLoader(val_ds,   batch_size=CONFIG['batch_size_cls'])

In [24]:
cls_model = xrv.models.DenseNet(weights='densenet121-res224-chex').to(device)
# Access classifier's in_features directly from cls_model.features
cls_model.classifier = torch.nn.Linear(cls_model.features.norm5.num_features, len(CHEXPERT_LABELS)).to(device)  # Replace 'model.classifier' with 'features.norm5'
# The following line is changed:
cls_model.features.conv0 = torch.nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).to(device) # Modify the first convolutional layer to accept 3 channels
# The following line is added to fix the error
cls_model.op_threshs = None # Reset op_threshs to avoid dimension mismatch

optimizer = torch.optim.AdamW(cls_model.parameters(), lr=CONFIG['lr_cls'])


for epoch in range(CONFIG['epochs_cls']):
    cls_model.train()
    total_loss = 0
    for imgs,labels in train_loader:
        imgs,labels = imgs.to(device), labels.to(device)
        preds = cls_model(imgs)
        loss = criterion(preds, labels)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{CONFIG['epochs_cls']} Loss: {total_loss/len(train_loader):.4f}")

# Validation
cls_model.eval()
y_t, y_p = [], []
with torch.no_grad():
    for imgs,labels in val_loader:
        out = torch.sigmoid(cls_model(imgs.to(device))).cpu().numpy()
        y_p.extend((out>0.5).astype(int)); y_t.extend(labels.numpy())

print(classification_report(y_t, y_p, target_names=CHEXPERT_LABELS, zero_division=0))

Epoch 1/3 Loss: 1.3595
Epoch 2/3 Loss: 1.2815
Epoch 3/3 Loss: 1.3274
                            precision    recall  f1-score   support

               Atelectasis       0.19      1.00      0.32        56
              Cardiomegaly       0.24      0.95      0.38        62
             Consolidation       0.01      1.00      0.02         3
                     Edema       0.02      1.00      0.05         7
                  Effusion       0.08      1.00      0.14        22
Enlarged Cardiomediastinum       0.00      0.00      0.00         0
                  Fracture       0.09      0.43      0.15        28
               Lung Lesion       0.24      0.66      0.35        53
              Lung Opacity       0.36      1.00      0.53       105
                No Finding       0.00      0.00      0.00        17
          Pleural Effusion       0.08      1.00      0.14        22
             Pleural Other       0.00      0.00      0.00         0
                 Pneumonia       0.02      1.0

In [25]:
# Save trained classification model
torch.save(cls_model.state_dict(), '/content/densenet_finetuned.pt')
print("Saved DenseNet model.")

Saved DenseNet model.


In [29]:
# Inference & Reporting
from peft import PeftModel

# Load text model
base_t5 = AutoModelForSeq2SeqLM.from_pretrained(CONFIG['t5_model'])
# Update the path to the LoRA model if necessary
lf_model = PeftModel.from_pretrained(
    base_t5,
    model_id="/content/drive/MyDrive/lora_models/t5_lora_model",
    is_local=True
).to(device)
tokenizer = AutoTokenizer.from_pretrained(CONFIG['t5_model'])

rouge = rouge_scorer.RougeScorer(['rouge1','rougeL'], use_stemmer=True)
bleu_fn = SmoothingFunction().method1

results = []
for _,row in df.iterrows():
    dcm_path = os.path.join(CONFIG['image_dir'],row.filename.replace('.png','.dcm'))
    ds = pydicom.dcmread(dcm_path)
    img = (torch.tensor(ds.pixel_array.astype(np.float32)).unsqueeze(0)-ds.pixel_array.mean())/ds.pixel_array.std()
    img = img.unsqueeze(0).to(device)
    with torch.no_grad():
        logits = cls_model(img)
    probs = torch.sigmoid(logits)[0].cpu().numpy()
    preds = [lbl for i,lbl in enumerate(CHEXPERT_LABELS) if probs[i]>0.5]
    prompt0 = f"Findings: {', '.join(preds)}. Generate a radiology report."
    rpt0 = lf_model.generate(**tokenizer(prompt0, return_tensors='pt', truncation=True).to(device), max_length=256)
    text0 = tokenizer.decode(rpt0[0], skip_special_tokens=True)
    prompt1 = f"History: {row.indication_clean}. Findings: {', '.join(preds)}. Generate a refined radiology report."
    rpt1 = lf_model.generate(**tokenizer(prompt1, return_tensors='pt', truncation=True).to(device), max_length=256)
    text1 = tokenizer.decode(rpt1[0], skip_special_tokens=True)
    r1 = rouge.score(row.impression_clean, text0)['rouge1'].fmeasure
    b1 = sentence_bleu([row.impression_clean.split()], text0.split(), smoothing_function=bleu_fn)
    r2 = rouge.score(row.impression_clean, text1)['rouge1'].fmeasure
    b2 = sentence_bleu([row.impression_clean.split()], text1.split(), smoothing_function=bleu_fn)
    results.append({
        'uid': row.uid,
        'predicted_labels': ';'.join(preds),
        'report_initial': text0,
        'report_refined': text1,
        'rouge1_init': r1,
        'rouge1_ref': r2,
        'bleu_init': b1,
        'bleu_ref': b2
    })

pd.DataFrame(results).to_csv(CONFIG['output_csv'], index=False)
print(f"Saved results to {CONFIG['output_csv']}")

ValueError: Can't find 'adapter_config.json' at '/content/drive/MyDrive/lora_models/t5_lora_model'

## 🔥 Fine-tune Flan-T5-small with LoRA (NEWLY ADDED)

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Reload Flan-T5 base model
base_t5 = AutoModelForSeq2SeqLM.from_pretrained(CONFIG['t5_model'])
tokenizer = AutoTokenizer.from_pretrained(CONFIG['t5_model'])

# Apply LoRA
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=16)
lf_model = get_peft_model(base_t5, peft_config).to(device)

# Prepare dataset for text fine-tuning
txt_ds = XRayDataset(df, len(df), mode='txt')

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/output",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_dir="/content/logs",
    save_strategy="no"
)

# Fine-tune
trainer = Seq2SeqTrainer(
    model=lf_model,
    args=training_args,
    train_dataset=txt_ds,
    eval_dataset=txt_ds
)
trainer.train()

# Save LoRA model
lf_model.save_pretrained(CONFIG['lora_save_dir'])
print(f"✅ LoRA model saved at {CONFIG['lora_save_dir']}")
